In [3]:
import pandas as pd

In [4]:
# 依次读入数据并查看缺失值情况
hyinfo_df = pd.read_excel('附件1-会员信息表.xlsx',sheet_name=0)

In [5]:
hyinfo_df.isna().sum()

kh          0
kh.1        0
csny    34188
xb       9435
djsj    12684
dtype: int64

In [6]:
xsls_df = pd.read_excel('附件2-销售流水表.xlsx')
xsls_df.isna().sum()

djh      0
spbm     0
sj       0
sl       0
je       0
dtime    0
syjh     0
dtype: int64

In [7]:
hyxf_df = pd.read_excel('附件3-会员消费明细表.xlsx')
hyxf_df.isna().sum()

kh           0
dtime        0
spbm         0
sl           0
sj           0
je           0
spmc         0
jf           0
syjh         0
djh          0
gzbm         0
gzmc     20478
dtype: int64

In [8]:
spinfo_df = pd.read_excel('附件4-商品信息表.xlsx')
spinfo_df.isna().sum()

spbm     0
spmc     0
splm    10
ppbm     0
lbbm     0
jxs      0
xxs      0
wsjj     0
hsjj     0
sj       0
jlrq     1
csbm     0
gzbm     2
dtype: int64

In [9]:
# 删除spinfo_df中有空值的行
nullidx = spinfo_df[spinfo_df.isna().any(axis=1)].index

In [10]:
spinfo_df = spinfo_df.drop(index=nullidx)

In [11]:
# 连接该店会员信息表和会员消费表
df1 = hyinfo_df.merge(hyxf_df, on='kh')

In [12]:
df1.columns

Index(['kh', 'kh.1', 'csny', 'xb', 'djsj', 'dtime', 'spbm', 'sl', 'sj', 'je',
       'spmc', 'jf', 'syjh', 'djh', 'gzbm', 'gzmc'],
      dtype='object')

In [13]:
# 删除重复的卡号列，收音机号，柜组名称（太多缺失）
df1 = df1.drop(columns=['kh.1', 'syjh', 'gzmc'])

In [14]:
# 增加优惠列，用售价减去消费金额
df1['yh'] = df1['sj'] - df1['je']

In [16]:
# 计算年龄
df1['csny'] = df1['csny'].astype('datetime64[Y]')
df1['nl'] = [(2018 - date.year) for date in list(df1['csny'])]

In [17]:
df1 = df1.drop(columns=['csny'])

In [23]:
# 计算消费时间
df1['dtime'] = df1['dtime'].astype('datetime64')
df1['dayofweek'] = df1['dtime'].dt.dayofweek
df1['hour'] = df1['dtime'].dt.hour
df1['dayofyear'] = df1['dtime'].dt.dayofyear

In [24]:
df1

,kh,xb,djsj,dtime,spbm,sl,sj,je,spmc,jf,djh,gzbm,yh,nl,dayofweek,hour
0,a37cc182,0.0,2004-12-31 21:24:34.216,2016-09-25 10:49:14.016,d62a69e3,1,12288.0,11089.93,WMF D无,11089.92,7cd8,7296,1198.07,51.0,6,10
1,a37cc182,0.0,2004-12-31 21:24:34.216,2016-09-25 10:49:14.016,d62a69e3,1,499.0,297.07,WMF D无,297.08,7cd8,7296,201.93,51.0,6,10
2,a37cc182,0.0,2004-12-31 21:24:34.216,2017-02-28 17:17:35.533,252403ef,1,598.0,598.00,双立人商品 F无,598.00,ed0a,7242,0.00,51.0,1,17
3,34c2dc93,0.0,2007-04-19 00:00:00.000,2017-11-13 13:36:45.656,4708e0bb,1,260.0,260.00,植村秀三色眼影/眼线液/眉笔.,260.00,c3f4,8156,0.00,32.0,0,13
4,828aa9b4,0.0,2004-09-24 16:39:08.716,2017-05-17 11:30:39.080,a4809307,1,1288.0,1288.00,酷彩F件,1288.00,95b0,7300,0.00,67.0,2,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542630,978bf4cc,0.0,2012-12-17 00:00:00.000,2017-11-26 17:54:05.063,36d50bfb,1,8096.0,7934.08,浪琴.,1587.00,8a3f,8078,161.92,35.0,6,17
542631,7adcdd4e,0.0,2013-08-30 00:00:00.000,2017-01-18 17:35:30.533,886d14dc,1,498.0,498.00,双立人商品系列无,498.00,ba1d,7242,0.00,46.0,2,17
542632,5cf06746,0.0,2013-08-30 00:00:00.000,2017-10-29 13:47:11.046,083ad3b7,1,2880.0,2880.00,威可多 F.,2880.00,25bb,5445,0.00,45.0,6,13
542633,5cf06746,0.0,2013-08-30 00:00:00.000,2017-10-29 13:47:11.046,083ad3b7,1,1320.0,1320.00,威可多 F.,1320.00,25bb,5445,0.00,45.0,6,13


In [32]:
# 保存为二进制的pickle格式，方便下次快速读取
import pickle
with open('（本店）会员信息和会员消费.pkl', 'wb') as f:
    pickle.dump(df1, f)

In [38]:
# 非会员数据是从消费流水中删除会员消费的单据号
hydjh = list(hyxf_df['djh'])
hydjhidx = []
for i,row in xsls_df.iterrows():
    if row['djh'] in hydjh:
        hydjhidx.append(i)

In [43]:
df2 = xsls_df.drop(index=hydjhidx)

In [1]:
df2['dtime'] = df2['dtime'].astype('datetime64')
df2['dayofweek'] = df2['dtime'].dt.dayofweek
df2['hour'] = df2['dtime'].dt.hour
df2['dayofyear'] = df2['dtime'].dt.dayofyear

NameError: name 'df2' is not defined

In [49]:
df2 = df2.drop(columns='syjh')

In [51]:
df2['yh'] =  df2['sj'] - df2['je']

In [52]:
# 保存pickle
with open('非会员消费.pkl', 'wb') as f:
    pickle.dump(df2, f)